In [2]:
import pandas as pd
import numpy as np
import math

weather = pd.read_csv('../data/input/weather.csv')
train = pd.read_csv('../data/input/train.csv')
test = pd.read_csv('../data/input/test.csv')
spray = pd.read_csv('../data/input/spray.csv')

In [3]:
# reducing the list of columns in weather
weather_excluded = ['Depth', 'Water1', 'SnowFall', 'Depart', 'Heat', 'Cool', 'Sunrise', 'Sunset']
weather_keep = [column for column in weather.columns if column not in weather_excluded]
weather = weather[weather_keep]

In [4]:
# dates to date objects
weather.Date = pd.DatetimeIndex(weather.Date)
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

In [5]:
# to remove the leading spaces
weather.PrecipTotal = weather.PrecipTotal.str.strip()

#### Looking at daily data, some dates have an "M" or a &"T" in the field. What does this mean?
"M" stands for "Missing". Data for an element will be missing if the primary sensor for that weather element is inoperable (e.g., has an outage) or malfunctioning (e.g., producing errant data) AND any collocated backup sensor is also inoperable or malfunctioning. "T" stand for "Trace". This is a small amount of precipitation that will wet a raingage but is less than the 0.01 inch measuring limit.

In [6]:
weather = weather.replace('T', 0.005)
weather = weather.replace('M', np.nan)

# Train formatting

In [7]:
# Train
train.Date = pd.to_datetime(train.Date)
cols = [i for i in train.columns]
for i,v in enumerate(cols):
    if '_date-1_date-' in v: # drops out the repeated date column
        train.drop(v,axis=1,inplace=True)

In [8]:
'''
Formula for calculating compass bearing between two lat/lon tuples.  Credit:  https://gist.github.com/jeromer/2005586
Corrected output error in which returned bearing (in degrees) needed to be subtracted from 360 in order to be correct.  
Validated this change on https://www.sunearthtools.com/tools/distance.php and with manual orienteering using Google 
maps in place of a physical map.

Other than that change at the end of the formula, the code was taken from the gist page linked above.
'''


def compass_bearing(loc1, loc2):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `loc1: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `loc2: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(loc1) != tuple) or (type(loc2) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(loc1[0])
    lat2 = math.radians(loc2[0])

    diffLong = math.radians(loc1[1] - loc2[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = 360 - ((initial_bearing + 360) % 360) ## Mike: this originally returned an incorrect bearing
                         ## corrected by subtracting result from 360

    return compass_bearing

''' 
Calculate distance in KM between two sets of coordinates (lat/lon tuples).  Uses Haversine formula.  Credit to https://gist.github.com/rochacbruno/2883505 and http://www.movable-type.co.uk/scripts/latlong.html

This formula works as-is.  I tested it using Google Maps distance calculator as a validator.
'''

def distance(loc1, loc2):
#     print (loc1, loc2)
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    radius = 6371 # radius of Earth in KM

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [9]:
"""
Mapping of each trap to its nearest weather station--compares distance in km between a trap and each airport weather station, and returns the station number ('Station' column) of the nearest weather station.

Weather station info from Kaggle:

Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

"""

station_coords = {
        1: (41.995,-87.933),
           2: (41.786,-87.752)}

train['Coordinates'] = train[['Latitude', 'Longitude']].apply(tuple, axis=1)

def assign_station(i):
    if distance(station_coords[1], i) < distance(station_coords[2], i):
        return 1
    else:
        return 2

train['Weather_Station'] = train.Coordinates.apply(assign_station)


In [10]:
train.loc[train['Coordinates'] == (41.992478000000006, -87.862994999999998), 'Trap'] = 'T009Alt'

# Back to Weather

In [11]:
weather_types = {
'\+FC': 'TORNADO/WATERSPOUT','FC': 'FUNNEL CLOUD','TS': 'THUNDERSTORM','GR': 'HAIL','RA': 'RAIN',
'DZ': 'DRIZZLE','SN': 'SNOW','SG': 'SNOW GRAINS','GS': 'SMALL HAIL ANDOR SNOW PELLETS','PL': 'ICE PELLETS',
'IC': 'ICE CRYSTALS','FG\+': 'HEAVY FOG','FG': 'FOG','BR': 'MIST','UP': "UNKNOWN PRECIPITATION",'HZ': 'HAZE','FU': 'SMOKE',
'VA': 'VOLCANIC ASH','DU': 'WIDESPREAD DUST','DS': 'DUSTSTORM','PO': 'SAND_DUST WHIRLS',
'SA': 'SAND','SS': 'SANDSTORM','PY': 'SPRAY','SQ': 'SQUALL','DR': 'LOW DRIFTING','SH': 'SHOWER','FZ': 'FREEZING',
'MI': 'SHALLOW','PR': 'PARTIAL','BC': 'PATCHES','BL': 'BLOWING','VC': 'VICINITY'}

for i in weather_types:
    weather[i] = 0
    weather.loc[weather.CodeSum.str.contains(i) == True,i] = 1

weather.drop('CodeSum',axis=1,inplace=True)

In [12]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,SeaLevel,...,MI,TS,SH,RA,\+FC,SA,SG,PO,PL,SN
0,1,2007-05-01,83,50,67,51,56,0.00,29.10,29.82,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-01,84,52,68,51,57,0.00,29.18,29.82,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-02,59,42,51,42,47,0.00,29.38,30.09,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-02,60,43,52,42,47,0.00,29.44,30.08,...,0,0,0,0,0,0,0,0,0,0
4,1,2007-05-03,66,46,56,40,48,0.00,29.39,30.12,...,0,0,0,0,0,0,0,0,0,0


In [14]:
n_weather = weather.iloc[12:,:]
for i in range(1,7): # Hard-coded range of the last 6 days
    n_date = "_date-"+str(i)
    n_weather.loc[:,n_date] = n_weather.Date-pd.DateOffset(i)
    n_weather = pd.merge(left=n_weather,right=weather,left_on=[n_date,'Station'],right_on=['Date','Station'],suffixes =('',n_date))

/Users/austinbrian/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/austinbrian/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
n_weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,SeaLevel,...,MI_date-6,TS_date-6,SH_date-6,RA_date-6,\+FC_date-6,SA_date-6,SG_date-6,PO_date-6,PL_date-6,SN_date-6
0,1,2007-05-07,83,47,65,41,54,0.005,29.38,30.12,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-07,84,50,67,39,53,0.00,29.44,30.12,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-08,82,54,68,58,62,0.00,29.29,30.03,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-08,80,60,70,57,63,0.005,29.36,30.02,...,0,0,0,0,0,0,0,0,0,0
4,1,2007-05-09,77,61,69,59,63,0.13,29.21,29.94,...,0,0,0,0,0,0,0,0,0,0


# Geography

In [16]:
# Creating de-duped df of traps and coordinates.
traps_master = train[['Trap', 'Coordinates']]
# Drop duplicates
traps_master.drop_duplicates(inplace=True)

/Users/austinbrian/anaconda/lib/python2.7/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [17]:
# Create list of column names to use with for loops below
trap_cols = traps_master.Trap.tolist() 

# Create dictionary of Trap IDs and corresponding lat/lon coordinate tuples
trap_dict = traps_master.set_index('Trap')['Coordinates'].to_dict()

In [18]:
# Creates dictionary where key is a Trap name, and the value for each is a list of the distances to every other trap.
distance_dict = {}
for i in trap_cols:
    dist_list = []
    for k in trap_dict:
        dist = distance(trap_dict[k], trap_dict[i])
        dist_list.append(dist)
    distance_dict[i] = dist_list


# Creates dictionary where key is a Trap name, and the value for each is a list of the compass bearings from every other trap.
bearing_dict = {}
for c in trap_cols:
    bearing_list = []
    for q in trap_dict:
        bearing = compass_bearing(trap_dict[q], trap_dict[c])
        bearing_list.append(bearing)
    bearing_dict[c] = bearing_list

In [19]:
# Creates df of relative distances.  To be merged with a left join into train df.
dist_df = pd.DataFrame.from_dict(distance_dict, orient='index')
distance_labels = dist_df.index.tolist()
distance_cols = []
for c in distance_labels:
    label = c + '_dist'
    distance_cols.append(label)
dist_df.columns = [distance_cols] 
dist_df['Trap'] = dist_df.index
# dist_df.to_csv('../data/relative_locations.csv')


# Creates df of relative bearings.  To be merged with a left join into train df.
bearing_df = pd.DataFrame.from_dict(bearing_dict, orient='index')  
bearing_labels = bearing_df.index.tolist()  
bearing_cols = []
for c in bearing_labels:
    label = c + '_bearing'
    bearing_cols.append(label)
bearing_df.columns = [bearing_cols]
bearing_df['Trap'] = bearing_df.index
# bearing_df.to_csv('../data/relative_bearings.csv')

In [20]:
train = pd.merge(train, dist_df, how='left', left_on='Trap', right_on='Trap')
train = pd.merge(train, bearing_df, how='left', left_on='Trap', right_on='Trap')

In [21]:
train.shape

(10506, 288)

In [22]:
n_weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,SeaLevel,...,MI_date-6,TS_date-6,SH_date-6,RA_date-6,\+FC_date-6,SA_date-6,SG_date-6,PO_date-6,PL_date-6,SN_date-6
0,1,2007-05-07,83,47,65,41,54,0.005,29.38,30.12,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-07,84,50,67,39,53,0.00,29.44,30.12,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-08,82,54,68,58,62,0.00,29.29,30.03,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-08,80,60,70,57,63,0.005,29.36,30.02,...,0,0,0,0,0,0,0,0,0,0
4,1,2007-05-09,77,61,69,59,63,0.13,29.21,29.94,...,0,0,0,0,0,0,0,0,0,0


In [23]:
train = pd.merge(train,n_weather,left_on=['Date','Weather_Station'],right_on=['Date','Station'])

In [27]:
train.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,\+FC_date-6,SA_date-6,SG_date-6,PO_date-6,PL_date-6,SN_date-6,Trap_Mosquitos_today,Trap_Species_today,min_Wnv_species_today,Epoch
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,0,0,0,0,0,0,2,1,0,1180396800


In [25]:
# grouping mosquitos together to get a more accurate picture of how many mosquitos in each trap
train['Trap_Mosquitos_today'] = train.groupby(['Trap','Date'])['NumMosquitos'].transform('sum')
train['Trap_Species_today'] = train.groupby(['Trap','Date','Species'])['NumMosquitos'].transform('sum')
train['min_Wnv_species_today'] = train.groupby(['Trap','Date','Species'])['WnvPresent'].transform('sum')

In [26]:
train['Epoch'] = train.Date.astype(np.int64) // 10**9

In [29]:
train.to_csv('~/Dropbox/DSI/train.csv')

In [63]:
# train.to_csv('../../../../datasets/Project4/workingfiles/train.csv',index=False)

-------
### Code graveyard

In [77]:
len(train_2.Trap)

10506

In [78]:
len(train_2.Species)

10506

In [79]:
train_2['Date_string'] = train_2.Date.astype(str)

In [88]:
# train_2['ObvID'] = pd.factorize(train_2.Date_string + train_2.Trap + train_2.Species)

In [69]:
rollup = train_2.pivot_table(index=['ObvID'], values=['NumMosquitos'], aggfunc=np.sum)

In [85]:
# rollup.head()